In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it


### Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [2]:
attendance_df = pd.read_csv('attendance.csv')
attendance_df.info()
attendance_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4 non-null      object
 1   2018-01-01  4 non-null      object
 2   2018-01-02  4 non-null      object
 3   2018-01-03  4 non-null      object
 4   2018-01-04  4 non-null      object
 5   2018-01-05  4 non-null      object
 6   2018-01-06  4 non-null      object
 7   2018-01-07  4 non-null      object
 8   2018-01-08  4 non-null      object
dtypes: object(9)
memory usage: 416.0+ bytes


,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attendance_df.rename(columns= {'Unnamed: 0': 'student'}, inplace=True)

In [4]:
attendance_T = attendance_df.T
attendance_T.rename(columns= {0: 'Sally', 1: 'Jane', 2: 'Billy', 3: 'John'}, inplace=True)
attendance_T = attendance_T.drop(['student'], axis=0)
attendance_T

attendance_T.info()
attendance_T

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 2018-01-01 to 2018-01-08
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Sally   8 non-null      object
 1   Jane    8 non-null      object
 2   Billy   8 non-null      object
 3   John    8 non-null      object
dtypes: object(4)
memory usage: 320.0+ bytes


,Sally,Jane,Billy,John
2018-01-01,P,A,A,P
2018-01-02,T,P,T,T
2018-01-03,T,T,A,H
2018-01-04,H,T,A,P
2018-01-05,P,T,H,P
2018-01-06,A,T,T,T
2018-01-07,T,A,P,P
2018-01-08,T,T,T,P


In [70]:
att_vals = {'A': 0, 'H': 0.5, 'P': 1, 'T': 0.9}
attendance_vals = attendance_T.replace(to_replace= att_vals, value=None)
attendance_vals

,Sally,Jane,Billy,John
2018-01-01,1.0,0.0,0.0,1.0
2018-01-02,0.9,1.0,0.9,0.9
2018-01-03,0.9,0.9,0.0,0.5
2018-01-04,0.5,0.9,0.0,1.0
2018-01-05,1.0,0.9,0.5,1.0
2018-01-06,0.0,0.9,0.9,0.9
2018-01-07,0.9,0.0,1.0,1.0
2018-01-08,0.9,0.9,0.9,1.0


In [71]:
print(attendance_vals.Sally.sum() / 8)
print(attendance_vals.Jane.sum() / 8)
print(attendance_vals.Billy.sum() / 8)
print(attendance_vals.John.sum() / 8)
# The numbers aren't right but at least half the battle is won...

0.7625
0.6875
0.525
0.9125


### Coffee Levels

Read the coffee_levels table.

Transform the data so that each carafe is in it's own column.

Is this the best shape for the data?

In [20]:
coffee_df = pd.read_csv('coffee_levels.csv')
coffee_df.info()
coffee_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hour           30 non-null     int64  
 1   coffee_carafe  30 non-null     object 
 2   coffee_amount  30 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 848.0+ bytes


,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [31]:
coffee_piv = pd.pivot_table(data= coffee_df, values= 'coffee_amount', index= 'hour', columns= 'coffee_carafe')
coffee_piv.columns.name = ''
coffee_piv = coffee_piv.reset_index()
coffee_piv.head()


,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947


In [38]:
coffee_repiv = coffee_piv.T
coffee_repiv.reset_index()
coffee_repiv.rename(columns= {0: '8:00', 1: '9:00', 2: '10:00', 3: '11:00', 4: '12:00', 5: '13:00', 6: '14:00', 7: '15:00', 8: '16:00', 9: '17:00'}, inplace=True)
coffee_repiv = coffee_repiv.drop(['hour'], axis=0)
coffee_repiv = coffee_repiv.reset_index()

,,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00
0,x,0.816164,0.451018,0.843279,0.335533,0.898291,0.310711,0.507288,0.215043,0.183891,0.391560
1,y,0.189297,0.521502,0.023163,0.235529,0.017009,0.997464,0.058361,0.144644,0.544676,0.594126
2,z,0.999264,0.915990,0.144928,0.311495,0.771947,0.398520,0.864464,0.436364,0.280621,0.436677


### Cake Recipes

Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

Tidy the data as necessary.

Which recipe, on average, is the best?  ($recipe b???$)

Which oven temperature, on average, produces the best results?  ($275???$)

Which combination of recipe, rack position, and temperature gives the best result?  ($recipe b, bottom rack, 300 degrees???$)

In [45]:
cake_df = pd.read_csv('cake_recipes.csv')
cake_df.info()
cake_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   recipe:position  8 non-null      object 
 1   225              8 non-null      float64
 2   250              8 non-null      float64
 3   275              8 non-null      float64
 4   300              8 non-null      float64
dtypes: float64(4), object(1)
memory usage: 448.0+ bytes


,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [ ]:
cake_df['recipe'] = cake_df['recipe:position'].str.split(':').str[0]
cake_df['position'] = cake_df['recipe:position'].str.split(':').str[1]
cake_df = cake_df.drop(columns= 'recipe:position')


In [66]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

In [ ]:
fallen_cake = cake_df.melt(id_vars= ['recipe', 'position'])
fallen_cake = fallen_cake.rename(columns= {'variable': 'temp', 'value': 'score'})
fallen_cake.sort_values(by= 'score', ascending=False)


In [72]:
fallen_cake.groupby('temp').mean()

,score
temp,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


In [75]:
pew_df = pd.read_csv('pew.csv')
pew_df.info()
pew_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   religion            18 non-null     object
 1   <$10k               18 non-null     int64 
 2   $10-20k             18 non-null     int64 
 3   $20-30k             18 non-null     int64 
 4   $30-40k             18 non-null     int64 
 5   $40-50k             18 non-null     int64 
 6   $50-75k             18 non-null     int64 
 7   $75-100k            18 non-null     int64 
 8   $100-150k           18 non-null     int64 
 9   >150k               18 non-null     int64 
 10  Don't know/refused  18 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 1.7+ KB


,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116
5,Evangelical Prot,575,869,1064,982,881,1486,949,723,414,1529
6,Hindu,1,9,7,9,11,34,47,48,54,37
7,Historically Black Prot,228,244,236,238,197,223,131,81,78,339
8,Jehovah's Witness,20,27,24,24,21,30,15,11,6,37
9,Jewish,19,19,25,25,30,95,69,87,151,162
